# Глава 10. Рубим, перемешиваем и нарезаем последовательбности

При реализации многомерного вектора, мы будетм польщоваться не наследованием, а *композицией*. Компненты веткора будут хранится в массиве ```array``` чисел с плавающей точкой, и мы напишем методы, необходимые для того чтобы ```Vector``` вел себя как неизменяемая плоская последовательность.

## ```Vector```, попытка №1: совместимость с ```Vector2d```

In [1]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __iter__(self):
        return iter(self._components)  # <2>

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6>

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>

* 1 В "защищенном" атрибуте экземпляра ```self.components``` хранится массив ```array``` компонент ```Vector```
* 2 Чтобы было возможно итерированние, возвращаем итератор, построенноый по ```self.components```
* 3 Используем ```reprlib.repr()``` для получения представления ```self._copmponents``` ограниченной длинны (например, ```array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...]```)
* 4 Удаляем префикс ```array```(```'d'``` и закрываем скобку), перед тем как подставить строку в вызов конструтора ```Vector```
* 5 Строим объект ```bytes``` из ```self._components```
* 6 Метод ```hypot``` больше не применим, поэтому вычисляем сумму квадратов компонент и извлекаем из нее квадратный корень
* 7 Единственное отличие от написанного ранее метода ```frombytes``` - последняя строка: мы передаем объект ```memoryview``` напрямую конструктору, не распаковывая его с помощью ```*```, как раньше

In [2]:
def reprs(_components):
        components = reprlib.repr(_components)  # <3>
        print(f'reprlib.repr(_components) = {components}')
        print(f'components.find("[") = {components.find("[")}')
        print(f'components[components.find("["):-1] = {components[components.find("["):-1]}')
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

_components = array('d', [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0])
reprs(_components)

reprlib.repr(_components) = array('d', [1.0, 2.0, 3.0, 4.0, 5.0, ...])
components.find("[") = 11
components[components.find("["):-1] = [1.0, 2.0, 3.0, 4.0, 5.0, ...]


'Vector([1.0, 2.0, 3.0, 4.0, 5.0, ...])'

Модуль ```reprlib``` обеспечивает средство для производства представлений объекта с ограничениями на размер получающегося строки. Он используется в отладчике Python и может быть полезно и в других контекстах.

Модуль предоставляет класс, сущность и функцию:

* ```class reprlib.Repr```
Класс, который предоставляет услуги форматирования, полезные в осуществлении функций, подобных встроенному repr(); добавляются ограничения по размеру для различных типов объектов, чтобы избежать создания чрезмерно длинных представлений.

* ```reprlib.aRepr```
Это - сущность Repr, который является используемый, чтобы обеспечить функцию repr(), описанную ниже. Изменение атрибуты этого объекта затронет пределы размера используемый repr() и отладчиком Python.

* ```reprlib.repr(obj)```
Это метод ```repr() aRepr```. Это возвращает подобное строка этому возвращенный встроенной функцией того же имени, но с ограничениями на большинство размеров.

В дополнение к средствам ограничения размера модуль также предоставляет декоратор для обнаружения рекурсивных вызовов ```__repr__()``` и замены местозаполнителя строка вместо него.

## Протоколы и динамическая типизация

Еще в главе 1 мы видели, что для создания полнофункционального типа последовательности в Python необязательно наследовать какому-то специальному классу;
нужно лишь реализовать методы, удовлетворяющие протоколу последовательности. Но что это за протокол такой?
В объектно-ориентированном программировании протоколом называется
неформальный интерфейс, определенный только в документации, но не в коде.
Например, протокол последовательности в Python подразумевает только наличие методов ```__len__``` и ```__getitem__```. Любой класс Spam, в котором есть такие
методы со стандартной сигнатурой и семантикой, можно использовать всюду,
где ожидается последовательность. Является Spam подклассом какого-то другого класса или нет, роли не играет.

In [3]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position] 

beer_card = Card (7, 'diamonds')
print(beer_card)
deck = FrenchDeck()
print(deck.suits)
print(deck[11::13])

Card(rank=7, suit='diamonds')
['spades', 'diamonds', 'clubs', 'hearts']
[Card(rank='K', suit='spades'), Card(rank='K', suit='diamonds'), Card(rank='K', suit='clubs'), Card(rank='K', suit='hearts')]


Любому опытному программисту на Python достаточно
одного взгляда на код, что понять, что это именно класс последовательности, несмотря на то, что он является подклассом ```object```. Мы говорим, что он является
последовательностью, потому что ведет себя, как последовательность, а только
это и важно.
Такой подход получил название **«динамическая типизация»**

## ```Vector```, попытка № 2: последовательность, допускающая срезку

B нашем случае таким атрибутом будет массив
self._components. Для начала нас вполне устроят такие однострочные методы```__len__``` и ```__getitem__```:
```
class Vector:
    # много строк опущено
    # ...
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        return self._components[index]
 ```

In [4]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR_V2
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]
# END VECTOR_V2

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [5]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [6]:
v1[0], v1[-1]

(3.0, 5.0)

In [7]:
v7 = Vector(range(7))
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [8]:
v7[0::2]

array('d', [0.0, 2.0, 4.0, 6.0])

In [9]:
v7[-1]

6.0

In [10]:
v7[1:4]

array('d', [1.0, 2.0, 3.0])

In [11]:
v7[-1:]

array('d', [6.0])

In [12]:
try:
    v7[1,2]
except Exception as err:
    print(err)
# TypeError

array indices must be integers


Как видите, даже срезы поддерживаются – но не очень хорошо. Было бы лучше,
если бы срез вектора также был экземпляром класса ```Vector```, а не массивом. В старом классе ```FrenchDeck``` была такая же проблема: срез оказывался объектом класса
```list```. Но в случае ```Vector``` мы утрачиваем значительную часть функциональности,
если операция среза возвращает простой массив.

## Как работает срезка

In [13]:
class MySeq:
    def __getitem__(self, index):
        return index # 1

s = MySeq()
s[1] # 2

1

In [14]:
s[1:4] # 3

slice(1, 4, None)

In [15]:
s[1:4:2] # 4

slice(1, 4, 2)

In [16]:
s[1:4:2, 9] # 5

(slice(1, 4, 2), 9)

In [17]:
s[1:4:2, 7:9] # 6

(slice(1, 4, 2), slice(7, 9, None))

* 1 Здесь ```__getitem__``` просто возвращает то, что ему передали
* 2 Один индекс, ничего нового
* 3 Нотация ```1:4``` преобразуется в ```slice(1, 4, None)```
* 4 ```slice(1, 4, 2)``` означает: начать с 1, закончить на 4, шаг 2
* 5 Сорприз: при наличии запятых внутри ```[]``` метод ```__getitem__``` получает кортеж
* 6 Этот кортеж может даже содержать несколько объектов среза

In [18]:
slice # 1

slice

In [19]:
dir(slice) # 2

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

* 1 ```slice``` - встроенный тип (мы это уже поняли в разделе "Объекты среза" главы 2)
* 2 Инспекция ```slice``` показывает наличие атрибутов ```start, stop, step```, и методода ```indeces```

In [20]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



```
help(slice.indices):
S.indices(len) -> (start, stop, stride)
```

В предположении, что длина последовательности равна len, вычисляет индексы start и stop, а также длину stride расширенного
среза, представленного объектом S. Индексы, выходящие за границы, приводятся к границам так же, как при обработке обычных
срезов.

Иначе говоря, метод ```indices``` раскрывает нетривиальную логику, применяемую
во встроенных последовательностях для корректной обработки отсутствующих
или отрицательных индексов и срезов, длина которых превышает длину конечной
последовательности. Этот метод возвращается «нормализованные» кортежи, содержащие неотрицательные целые числа ```start, stop, stride```, скорректированные
так, чтобы не выходить за границы последовательности заданной длины.

In [21]:
help(MySeq)


Help on class MySeq in module __main__:

class MySeq(builtins.object)
 |  Methods defined here:
 |  
 |  __getitem__(self, index)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [22]:
abc = 'ABCDE'
len(abc)

5

In [23]:
slice(None, 10, 2).indices(len(abc)) # 1 

(0, 5, 2)

In [24]:
slice(-3, None, None).indices(len(abc)) # 2

(2, 5, 1)

* 1 ```ABCDE[:10:2]``` - то же самое, что ```ABCDE[0:5:2]```
* 2 ```ABCDE[-3]``` - то же самое, что ```ABCDE[2:5:1]```

## Метод ```__getitem__``` с учетом срезов

In [25]:
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR_V2
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)  # <1>
        if isinstance(index, slice):  # <2>
            return cls(self._components[index])  # <3>
        elif isinstance(index, numbers.Integral):  # <4>
            return self._components[index]  # <5>
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))  # <6>
# END VECTOR_V2

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

* 1 Получаем класс экземпляра ```Vector```, он понадобится позже
* 2 Если аргумент ```index``` принадлежи типу ```slice```
* 3 ...то вызываем  класс для построения нового экземпляра ```Vector``` по срезу массива ```_components```...
* 4 Если ```index``` принадлежит типу ```int``` или другому целочисленому типу...
* 5 ...то просто возвращаем один конкретный элемент из ```_compomemts```
* 6 Иначе возбуждаем исключение 

```raise``` Возбуждает указанное исключение. Инструкция позволяет прервать штатный поток исполнения при помощи возбуждения исключения.

In [26]:
v7 = Vector(range(7))
v7[-1] # 1

6.0

In [27]:
v7[1:4] # 2

Vector([1.0, 2.0, 3.0])

In [28]:
v7[-1:] # 3

Vector([6.0])

In [29]:
try:
    v7[1,2]
except TypeError as err:
    print(err)

Vector indices must be integers


* 1 Если индекс - целое число, то извлекается ровно одна компонента типа ```float```
* 2 Если задан индекс типа ```slice```, то создается новый объект ```Vector```
* 3 Если длина среза ```len == 1```, то все равно создается новый объект ```Vector```
* 4 Класс ```vector```не поддерживает многомерное использование индексирование, поэтому при задании кортежа индексов или срезов возбуждается исключение

## ```Vector```, попытка №3: доступ к динамическим атрибутам

При переходе от класса ```Vector2d``` к ```Vector``` мы потеряли возможность обращаться
к компонентам вектора по имени, например: ```v.x```, ```v.y```. Теперь мы имеем дело с векторами, имеющими сколь угодно много компонент. Тем не менее, иногда удобно
обращаться к нескольким первым компонентам по именам, состоящим из одной
буквы, например, ```x```, ```y```, ```z``` вместо ```v[0]```, ```v[1]``` и ```v[2]```.

```Python
>>> v = Vector(range(10))
>>> v.x
0.0
>>> v.y, v.z, v.t
(1.0, 2.0, 3.0)
```

В классе ```Vector2d``` мы предоставляли доступ для чтения компонент ```x``` и ```y``` с помощью декоратора ```@property```. Мы могли бы завести и в ```Vector``` четыре свойства, но это утомительно. Специальный метод ```__getattr__``` позволяет сделать
это по-другому и лучше.

In [30]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

# BEGIN VECTOR_V3_GETATTR
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # <1>
        if len(name) == 1:  # <2>
            pos = cls.shortcut_names.find(name)  # <3>
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))
# END VECTOR_V3_GETATTR

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

* 1 Получить и запомнить класс ```Vector```, он понадобится позже
* 2 Если имя состоит из одного символа, то этот символ может входить в строку ```shortcut_names```
* 3 Найти позицию символа, составляюего односимвольное имя; метод ```str.find``` нашел бы точно такую же строку ```'yz'```, но нам это нe нужно, отсюда и  дополнительная проверка строчкой выше
* 4 Если символ найден, вернуть эелемент массива
* 5 Если предидущая проверка не прошла, возбудить исключение ```AttributeError``` со стандартным собщением

In [31]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [32]:
v.x # 1

0.0

In [33]:
v.x = 10 # 2
v.x # 3

10

In [34]:
v # 4

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

Реализовать метод ```__getattr__``` просто, но в данном случае недостаточно.
Неправильное поведение: присваивание v.x не приводит к ошибке, но результат получается несогласованным

* 1 Доступ к элементу ```v[0]``` по имени ```v.x```
* 2 Присваиваем ```v.x``` новое значение. При этом должно бы возникнуть исключение
* 3 Чтение ```v.x``` показывает новое значение, 10
* 4 Онако компоненты вектора не изменились

Метод ```__getattr__``` вызывается интерпретатором, если поиск атрибута завершается неудачно. Иначе говоря, анализируя выражение ```my_obj.x```, Python проверяет, есть ли у объекта ```my_obj``` атрибут с именем ```x```; если нет, поиск повторяется в классе ```(my_obj.__class__)```, а затем вверх по иерархии наследования3
. Если
атрибут ```x``` все равно не найден, то вызывается метод ```__getattr__```, определенный
в классе ```my_obj```, причем ему передается ```self``` и имя атрибута в виде строки (например, ```'x'```).
В примере приведен код метода ```__getattr__```. Он проверяет, является ли
искомый атрибут одной из букв ```xyzt```, и, если да, то возвращает соответствующую
компоненту вектора.

Несогласованность возникла из-за способа работы ```__getattr__```:
Python вызывает этот метод только в том случае, когда у объекта нет атрибута
с указанным именем. Однако же после присваивания ```v.x = 10``` у объекта v появился атрибут ```x```, поэтому ```__getattr__``` больше не вызывается для доступа к ```v.x```:
интерпретатор просто вернет значение *10*, связанное с ```v.x```. С другой стороны, в
реализации ```__getattr__``` мы игнорируем все атрибуты экземпляра, кроме ```self.
_components```, откуда читаются значения «виртуальных атрибутов», перечисленных
в строке ```shortcut_names```.

Напомним, что в последних вариантах класса ```Vector2d``` в главе 9 попытка
присвоить значение атрибутам экземпляра ```.x``` или ```.y``` приводила к исключению
```AttributeError```. В классе ```Vector``` мы хотим возбуждать такое же исключение при
любой попытке присвоить значение атрибуту с однобуквенным именем – просто
во избежание недоразумений. Для этого реализуем метод ```__setattr__```.

In [35]:
class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))

# BEGIN VECTOR_V3_SETATTR
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # <1>
            if name in cls.shortcut_names:  # <2>
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3>
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # <4>
            if error:  # <5>
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # <6>

# END VECTOR_V3_SETATTR

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

* 1 Специальная обработка односимвольных имен атрибутов
* 2 Если имя совпадает с одним их символов ```xyzt```, задать один текст сообщения об ошибке
* 3 Еслм имя - строчная буква, задать другой текст сообщения - обо всех однобуквенных именах
* 4 В противном случае оставить сообщение об ошибке пустым
* 5 Если сообщение об ошибке не пусто, возбуждаем исключение
* 6 Случай по умолчанию: вызвать метод ```__setatr__``` суперкласса для поолучения стандартного поведения

In [36]:
v = Vector(range(7))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [37]:
v.x

0.0

In [38]:
v.t

3.0

In [39]:
try:
    v.x = 10
except AttributeError as err:
    print(err)

readonly attribute 'x'


Если бы мы решили допустить изменение компонент, то могли бы реализовать метод ```__setitem__```, чтобы можно было писать ```v[0] = 1.1```, и (или) метод
```__setattr__```, чтобы работала конструкция ```v.x = 1.1```. Но сам класс ```Vector``` должен
оставаться неизменяемым, потому что в следующем разделе мы собираемся сделать его хэшируемым.

## ```Vector```, попытка №4: хэширование и ускорение оператора ```==```

И снова нам предстоит реализовать метод ```__hash__```. В сочетании с уже имеющимся методом ```__eq__``` это сделает экземпляры класса Vector хэшируемыми. Метод ```__hash__``` в примере ```9.1 Vetor2d``` просто вычислял выражение ```hash(self.x) ^ hash(self.y)```. Теперь мы хотели бы применить оператор **^ (ИСКЛЮЧАЮЩЕЕ ИЛИ)** к хэшам всех компонент: ```v[0] ^ v[1] ^ v[2]…``` . Тут нам на помощь придет функция ```functools.reduce```. Выше я говорил, что функция ```reduce``` уже не так популярна, как в былые времена, но для вычисления хэша всех компонент она
подходит идеально. На рис. 10.1_reduce_1 и 10.1_reduce_2 представлена общая идея функции ```reduce```.

![body](img/10.1_reduce_2.png)
![body](img/10.1_reduce_1.png)

Редуцирующие функции – ```reduce, sum, any, all``` – порождают единственное значение-агрегат из последовательности или произвольного конечного итерируемого объекта

До сих пор мы видели, что функцию ```functools.reduce()``` можно заменить функцией ```sum()```, а теперь объясним, как же она все-таки работает. Идея в том, чтобы редуцировать последовательность значений в единственное значение. Первый
аргумент ```reduce()``` – функция с двумя аргументами, а второй – итерируемый объект. Допустим, что имеется функция с двумя аргументами fn и список ```lst```. Если написать ```reduce(fn, lst)```, то ```fn``` сначала применяется к первым двум элементам – ```fn(lst[0], lst[1])``` – и в результате получится первый результат ```r1```. Затем ```fn``` применяется к``` r1``` и следующему элементу – ```fn(r1, lst[2]);``` так мы получаем второй результат ```r2```. Затем вызов ```fn(r2, lst[3])``` порождает ```r3``` … и так далее до последнего
элемента, после чего возвращается окончательный результат ```rN```. Вот как можно было бы применить ```reduce``` для вычисления 5! (факториал 5):

In [40]:
2 * 3 * 4 * 5 #  Ожтдаемый результат 5!=120

120

In [41]:
import functools
functools.reduce(lambda a,b: a*b, range(1, 6))

120

Но вернемся к проблеме хэширования. В примере показано, как можно было бы вычислить результат многократного применения ^ тремя способами:
один – с помощью цикла ```for``` и два – с помощью ```reduce```.

In [42]:
n = 0
for i in range(1, 6): # 1
    n ^= i

n

1

In [43]:
functools.reduce(lambda a,b: a^b, range(1, 6)) # 2

1

In [44]:
import operator
functools.reduce(operator.xor, range(1, 6)) # 3

1

* 1 Агрегирование в цикле ```for``` в накопительную переменную
* 2 ```functools.reduce``` с анонимной функцией
* 3 ```functools.reduce``` с заменой специального написного лямбда-выражения функцией ```operator.xor``` 

In [45]:
from array import array
import reprlib
import math
import numbers
import functools # 1
import operator # 2 


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other) # 3

    def __hash__(self):
        hashes = (hash(x) for x in self._components) # 4
        return functools.reduce(operator.xor, hashes, 0) # 5

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

* 1 Импортируем ```functools``` для использования ```reduce```
* 2 Импортируем ```operator``` для использования ```xor```
* 3 Метод ```__eq__``` не изменился; я привел его, только  потому что методы ```__eq__``` и ```__hash__``` принято распологать в коде рядом т.к. они дополняют друг друга
* 4 Создаем генераторное выражение для отложеного вычисления хэша каждой компоненты
* Подаем выражение ```hashes``` на вход вместе с функцией ```xor``` - для вычисления итогового хэш-значения; третий аргумент, равный 0, - **инициализатор**

При использовании ```reduce``` рекомендуется задавать третий аргумент, ```reduce(function, iterable, initializer)```, чтобы предотвратить появление исключения ```TypeError: reduce() of empty sequence with no initial value``` (отличное сообщение: описывается проблема и способ исправления). Значение ```initializer``` возвращается, если последовательность пуста, а, кроме того,
используется в качестве первого аргумента в цикле редукции,
поэтому оно должно быть нейтральным элементом относительно выполняемой операции. Так, для операций ```+, |, ^ initializer```
должен быть равен 0, а для ```*, & – 1```.

Метод ```__hash__ ```в данном примере – отличный пример техники **mapreduce**

![title](img/10.1_reduce_map.png)

**Map-reduce**: применить функцию к каждому элементу
для генерации новой последовательности ```map```,
затем вычислить агрегат ```reduce```

На шаге отображения (```map```) порождается один хэш для каждого компонента, а
на шаге редукции (```reduce```) все хэши агрегируются с помощью оператора ```xor```. Если
использовать функцию ```map``` вместо генераторного выражения, то шаг отображения
станет даже более наглядным:

```
def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)
```

Решение на основе map не так эффективно в Python 2, где функция map строит список, содержащий результаты. Однако в Python 3
map откладывает вычисления: она порождает генератор, который
отдает результаты по требованию, экономя тем самым память, –
точно так же, как генераторное выражение в методе ```__hash__``` из
примера

```
def __eq__(self, other):
    return tuple(self) == tuple(other)
```
Она работает для ```Vector2d``` и для ```Vector``` – и даже считает, что ```Vector([1, 2])```
равен (1, 2); это может оказаться проблемой, но пока закроем на нее глаза5
. Но для
векторов с тысячами компонент эта реализация крайне неэффективна. Она строит
два кортежа, полностью копируя оба операнда, только для того, чтобы воспользоваться оператором ```__eq__``` из типа ```tuple```. Такая экономия усилий вполне оправдана
для класса ```Vector2d``` (всего с двумя компонентами), но не для многомерных век- компонентами), но не для многомерных век- ами), но не для многомерных векторов. Более эффективный способ сравнения объекта ```Vector``` с другим объектом
```Vector```

In [46]:
def __eq__(self, other):
    if len(self) != len(other): # 1
        return False
    for a, b in zip(self, other): # 2 
        if a != b: # 3
            return False
    return True # 4

* 1 Если длины объектов различны, то они не равны
* 2 Функция ```zip``` порождает генератор кортежей, содержащих соответственные элементы каждого переданного ей итерируемого объекта. Если вы с
ней незнакомы, см. врезку «Удивительная функция ```zip```» ниже. Сравнение
длин в предыдущем предложении необходимо, потому что ```zip``` без предупреждения перестает порождать значения, как только хотя бы один входной аргумент оказывается исчерпанным.
* 3 Как только встречаются две различных компоненты, выходим и возвращая - компоненты, выходим и возвращаем ```False```.
* 4 В противном случае объекты равны.

### Удивительная функция ```zip```

Наличие цикла ```for```, в котором можно обойти элементы коллекции
без возни с индексной переменной, – отличное дело, и многие ошибки
так можно предотвратить, только для этого нужны специальные служебные функции. Одна из них – встроенная функция ```zip```, позволяющая
параллельно обходить два и более итерируемых объекта: она возвращает кортежи, которые можно распаковать в переменные, – по одной для
каждого входного объекта.

In [98]:
zip(range(3), 'ABC') # 1

In [99]:
list(zip(range(3), 'ABC')) # 2

[(0, 'A'), (1, 'B'), (2, 'C')]

In [100]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) # 3

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [101]:
from itertools import zip_longest # 4
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (None, None, 3.3)]

In [102]:
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

* 1 ```zip``` возвращает генератор, который порождает кортежи по запросу.
* 2 Здесь мы строим из генератора список ```list``` просто для отображения; обычно генератор обходят в цикле.
* 3 У ```zip``` есть удивительное свойство: она останавливается, не выдавая предупреждения, как только один из итерируемых объектов
оказывается исчерпанным.
* 4 Функция ```itertools.zip_longest``` ведет себя иначе: она подставляет вместо отсутствующих значений необязательный аргумент
```fillvalue``` (по умолчанию ```None```), поэтому генерирует кортежи,
пока не окажется исчерпанным самый длинный итерируемый
объект.

In [103]:
for a, b in zip(range(3), range(3)):
    if a == b:
        print(f'{a} == {b} ? {a==b}')

0 == 0 ? True
1 == 1 ? True
2 == 2 ? True


## ```Vector```, попытка №5: форматирование

In [104]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):  # <2>
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # <3>
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # <4>
            outer_fmt = '<{}>'  # <5>
        else:
            coords = self
            outer_fmt = '({})'  # <6>
        components = (format(c, fmt_spec) for c in coords)  # <7>
        return outer_fmt.format(', '.join(components))  # <8>

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

* 1 Импортируем itertools, чтобы можно было воспользоваться функцией
chain в методе ```__format__```.
* 2 Вычисляем одну из угловых координат по формулам, взятым из статьи по
адресу http://en.wikipedia.org/wiki/N-sphere.
* 3 Создаем генераторное выражение для вычисления всех угловых координат
по запросу.
* 4 Используем ```itertools.chain``` для порождения генераторного выражения,
которое перебирает модуль и угловые координаты вектора.
* 5 Конфигурируем отображение сферических координат в угловых скобках.
* 6 Конфигурируем отображение декартовых координат в круглых скобках.
* 7 Создаем генераторное выражение для форматирования координат по запросу.
	 Подставляем отформатированные компоненты, разделенные запятыми, в
угловые или круглые скобки.